In [2]:
# Import modules
import tensorflow as tf
import keras as K
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub
from keras.models import load_model

import pandas as pd
import numpy as np
import PIL
import logging
import os

import zipfile
import shutil

from datetime import datetime

# for figures
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns

# for the Grad-CAMs
from IPython.display import Image, display
import matplotlib.cm as cm

# Working Directory on A4 Computer (GoPro and V2 data are on the A4 Computer)
files = os.listdir()

# os.chdir("/home/scottweichenthal/Dropbox/IMAGINE Project/MSSI_Project/")
os.chdir("/home/scott/Dropbox/IMAGINE Project/MSSI_Project/")
cwd = os.getcwd()
print("Current working directory:", cwd)

# This code allows GPU memory allocation to grow as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
# filter out INFO logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Current working directory: /home/scott/Dropbox/IMAGINE Project/MSSI_Project


In [3]:
#  Prints the Tensorflow, Keras versions, and number of GPUs
print(tf.__version__)
print(K.__version__)
print("TensorFlow Hub version:", hub.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.14.0
2.14.0
TensorFlow Hub version: 0.14.0
Num GPUs Available:  4


In [4]:
from platform import python_version

print(python_version())
print("cuDNN version:", tf.sysconfig.get_build_info()['cudnn_version'])

3.9.19
cuDNN version: 8


In [5]:
!nvidia-smi

Mon Aug  5 16:17:40 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     On   | 00000000:05:00.0  On |                  N/A |
| 23%   39C    P8    11W / 250W |    234MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN Xp     On   | 00000000:06:00.0 Off |                  N/A |
| 24%   

In [7]:
# Ensure there's enough space
def check_disk_space():
    total, used, free = shutil.disk_usage("/")
    print(f"Total: {total // (2**30)} GiB")
    print(f"Used: {used // (2**30)} GiB")
    print(f"Free: {free // (2**30)} GiB")
    return free

free_space = check_disk_space()
if free_space < (1 * 2**30):  # Check if free space is less than 1 GiB
    print("Warning: Low disk space. Consider freeing up some space.")

Total: 3666 GiB
Used: 1022 GiB
Free: 2457 GiB


In [8]:
# Load data from test site
metadata = pd.read_csv('compiled data/rishabh/Data_estimation_experiments/test_site_metadata_site_id_2_with_data_splits.csv')

metadata.head(10)

,datetime,image_path,audio_path,ln_ufp_num_10s_ma_image_label_raw,ln_ufp_num_10s_ma_spec_label_raw,ln_ufp_size_10s_ma_image_label_raw,ln_ufp_size_10s_ma_spec_label_raw,ln_noise_10s_ma_image_label_raw,ln_noise_10s_ma_spec_label_raw,ufp_num_10s_ma_image_label_raw,...,image_extension,audio_extension,image_name,audio_name,pair_pm25,set_V3ext,set,vsby_nm,expt_set_number,train_val
0,2022-01-28T19:46:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.460771,8.400569,3.238678,3.222868,4.174695,4.296741,4725.7,...,.jpg,.png,220128_1446.jpg,220128_1446.png,3.91875,trn,trn,30.0,set_1,train
1,2022-01-28T19:47:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.119160,8.125069,3.310543,3.284664,4.036715,4.187835,3358.2,...,.jpg,.png,220128_1447.jpg,220128_1447.png,3.91875,trn,trn,30.0,set_1,train
2,2022-01-28T19:48:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.036897,7.893759,3.306887,3.430756,3.980055,3.910422,3093.0,...,.jpg,.png,220128_1448.jpg,220128_1448.png,3.91875,val,val,30.0,set_1,train
3,2022-01-28T19:49:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,7.991119,8.082618,3.367296,3.265759,3.933784,3.924347,2954.6,...,.jpg,.png,220128_1449.jpg,220128_1449.png,3.91875,trn,trn,30.0,set_1,train
4,2022-01-28T19:50:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.069468,8.170836,3.222868,3.222868,4.134686,4.149937,3195.4,...,.jpg,.png,220128_1450.jpg,220128_1450.png,3.91875,val,val,30.0,set_1,train
5,2022-01-28T19:51:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.328667,10.172396,3.218876,3.218876,4.242477,4.192529,30597.3,...,.jpg,.png,220128_1451.jpg,220128_1451.png,3.91875,trn,trn,30.0,set_1,train
6,2022-01-28T19:52:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.082575,9.962553,2.639057,2.602690,4.083452,4.087656,23922.5,...,.jpg,.png,220128_1452.jpg,220128_1452.png,3.91875,trn,trn,30.0,set_1,train
7,2022-01-28T19:53:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.408543,9.307331,2.624669,2.694627,4.163093,4.145671,12192.1,...,.jpg,.png,220128_1453.jpg,220128_1453.png,3.91875,trn,trn,30.0,set_1,train
8,2022-01-28T19:54:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.872606,9.844661,2.833213,2.833213,4.177919,4.171460,19391.8,...,.jpg,.png,220128_1454.jpg,220128_1454.png,3.91875,trn,trn,30.0,set_1,train
9,2022-01-28T19:55:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.503771,9.560391,2.821379,2.734368,4.142817,4.163871,13410.2,...,.jpg,.png,220128_1455.jpg,220128_1455.png,3.91875,trn,trn,30.0,set_1,train


In [9]:
# # Create the 'expt_set_number' column with default value 'set_3'
# df['expt_set_number'] = 'set_3'

# # Label the first 5000 rows as 'set_1'
# df.loc[:4999, 'expt_set_number'] = 'set_1'

# # Label the next 10000 rows as 'set_2'
# df.loc[5000:14999, 'expt_set_number'] = 'set_2'

# # df['expt_set_number'].value_counts()

# # Save the DataFrame to a CSV file (optional)
# df.to_csv('compiled data/rishabh/Data_estimation_experiments/test_site_metadata_site_id_2_with_data_splits.csv', index=False)

In [7]:
# data = pd.read_csv('compiled data/rishabh/Data_estimation_experiments/test_site_metadata_site_id_2_with_data_splits.csv')

# # Function to split each set into train and val
# def split_train_val(group, train_ratio=0.8):
#     train_size = int(len(group) * train_ratio)
#     group['train_val'] = ['train'] * train_size + ['val'] * (len(group) - train_size)
#     return group

# # Apply the function to each group
# df = df.groupby('expt_set_number', group_keys=False).apply(split_train_val)

# # Print the result
# # print(df.head(20))
# # print(df.tail(20))
# # df.head(20)
# # df.tail(20)
# # df.iloc[32990:33010]

# # Save the modified DataFrame to a CSV file
# df.to_csv('compiled data/rishabh/Data_estimation_experiments/final_test_data.csv', index=False)

In [11]:
# How much data in trn, val, tst sets for metadata_random_split
# Use column `expt_set_number` for the following experiments

metadata.expt_set_number.value_counts()

expt_set_number
set_3    22500
set_2    10000
set_1     5000
Name: count, dtype: int64

In [10]:
metadata.columns

Index(['datetime', 'image_path', 'audio_path',
       'ln_ufp_num_10s_ma_image_label_raw', 'ln_ufp_num_10s_ma_spec_label_raw',
       'ln_ufp_size_10s_ma_image_label_raw',
       'ln_ufp_size_10s_ma_spec_label_raw', 'ln_noise_10s_ma_image_label_raw',
       'ln_noise_10s_ma_spec_label_raw', 'ufp_num_10s_ma_image_label_raw',
       'ufp_num_10s_ma_spec_label_raw', 'ufp_size_10s_ma_image_label_raw',
       'ufp_size_10s_ma_spec_label_raw', 'noise_10s_ma_image_label_raw',
       'noise_10s_ma_spec_label_raw', 'temp_airp', 'wspd_airp', 'year', 'city',
       'pm25', 'file_exists', 'hardware', 'site_id', 'device_id', 'file',
       'noise_10s_ma_spec_label_quartile', 'noise_10s_ma_image_label_quartile',
       'ufp_num_10s_ma_spec_label_quartile',
       'ufp_num_10s_ma_image_label_quartile',
       'ufp_size_10s_ma_spec_label_quartile',
       'ufp_size_10s_ma_image_label_quartile', 'image_extension',
       'audio_extension', 'image_name', 'audio_name', 'pair_pm25', 'set_V3ext',
       's

In [12]:
# Use column `train_val` to choose training and validation data for each data split
metadata.train_val.value_counts()

train_val
train    30000
val       7500
Name: count, dtype: int64

In [13]:
# Choose from the 3 different data splits (expt_set_number)
expt_set = input("Which experiment set do you want to use: set_1, set_2 or set_3 ?")

# Filter the metadata dataframe
filtered_metadata_train = metadata[(metadata['expt_set_number'] == expt_set) & (metadata['train_val'] == 'train')]
filtered_metadata_val = metadata[(metadata['expt_set_number'] == expt_set) & (metadata['train_val'] == 'val')]

print("Total data count in train set:", filtered_metadata_train.shape[0])
print("Total data count in val set:", filtered_metadata_val.shape[0])

Which experiment set do you want to use: set_1, set_2 or set_3 ?set_3
Total data count in train set: 18000
Total data count in val set: 4500


In [14]:
# Select Exposure to be modelled
import os
import keras as K
# from keras.preprocessing.image import ImageDataGenerator

cwd = os.getcwd()
print("Current working directory:", cwd)

# Preprocessing function
architecture_preprocessing = K.applications.convnext.preprocess_input
    
def check_input_and_target_compatibility(target):
    if ((target == 'ln_noise_10s_ma_image_label_raw' or 
         target == 'ln_ufp_num_10s_ma_image_label_raw' or
         target == 'ln_ufp_size_10s_ma_image_label_raw') and
        file == 'audio_path'):
        print("!!! Input file not compatible with exposure !!!")
        return False
        
    elif ((target == 'ln_noise_10s_ma_spec_label_raw' or 
           target == 'ln_ufp_num_10s_ma_spec_label_raw' or
           target == 'ln_ufp_size_10s_ma_spec_label_raw') and
          file == 'image_path'):
        print("!!! Input file not compatible with exposure !!!")
        return False
        
    else:
        return True

# Get train and validate generators
def get_train_val_generator(target):
    # Making sure that wrong input is not fed into target
    if not check_input_and_target_compatibility(target):
        print("Please make sure `target` is compatible with `file_path`")
        return
        
    # Define the ImageDataGenerator with preprocessing function
    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                         horizontal_flip=True,
                                                         vertical_flip=False)

    # Training generator - use filtered metadata
    train_generator = generator.flow_from_dataframe(dataframe=filtered_metadata_train[[target, file]].reset_index(drop=True),  
                                                    x_col=file,  # file = image_path or audio_path
                                                    y_col=target,  # target = exposure to be modelled
                                                    class_mode='raw', 
                                                    target_size=(256, 256),  # all of our images will be resized to 256 x 256
                                                    color_mode='rgb', 
                                                    batch_size=64,
                                                    shuffle=True)

    # Validation generator - use only the validation site
    validate_generator = generator.flow_from_dataframe(dataframe=filtered_metadata_val[[target, file]].reset_index(drop=True),
                                                       x_col=file,
                                                       y_col=target,
                                                       class_mode='raw',
                                                       target_size=(256, 256),
                                                       color_mode='rgb',
                                                       batch_size=64,
                                                       shuffle=False)
    

    # Define callbacks
    # Log file name (Change this if running multiple experiments on same model)
    
    csv_logger = K.callbacks.CSVLogger(f'./rishabh_model_files/logs/data_estimation_experiment/{expt_set},{target},convnext,{file},{meta_data_name}.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint(
        f'./rishabh_model_files/models/data_estimation_experiment/{expt_set},{target},convnext,{file},{meta_data_name}.tf',
        monitor='val_loss',
        mode='auto',
        save_weights_only=False,
        save_best_only=True
    )
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

    return train_generator, validate_generator, [csv_logger, model_checkpoint, reduce_lr_on_plateau, early_stopping]


Current working directory: /home/scott/Dropbox/IMAGINE Project/MSSI_Project


In [32]:
# Define global variables for dropout rate and L2 regularization strength
# global_dropout_rate = 0.5
# global_l2_regularization = 1e-5
# initial_learning_rate = 1e-5
# meta_data_name = "data_estimation_expt"

In [17]:
# Load the best model for UFP_size_image

meta_data_name = "data_estimation_expt"

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

input_model = input("Which pre-trained model do you want to load: ufp_size_image_model, ufp_size_spec_model, ufp_num_image_model, ufp_num_spec_model, noise_image_model, noise_spec_model ? ")

if input_model == 'ufp_size_image_model':
    with strategy.scope():
        # Load the best model for UFP_size_image
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_ufp_size_10s_ma_image_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,image_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.5_l2_1e-05_higher_dropout.tf')
        file = 'image_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_ufp_size_10s_ma_image_label_raw')
        print("best ufp_size_image_model loaded")
        
elif input_model == 'ufp_size_spec_model':
    with strategy.scope():
        # Load the best model for UFP_size_spectrogram
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_ufp_size_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.5_l2_0.0001_higher_dropout.tf')
        file = 'audio_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_ufp_size_10s_ma_spec_label_raw')
        print("best ufp_size_spec_model loaded")
        
elif input_model == 'ufp_num_image_model':
    with strategy.scope():
        # Load the best model for UFP_num_image
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_ufp_num_10s_ma_image_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,image_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.3_l2_0.0001.tf')
        file = 'image_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_ufp_num_10s_ma_image_label_raw')
        print("best ufp_num_image_model loaded")
        
elif input_model == 'ufp_num_spec_model':
    with strategy.scope():
        # Load the best model for UFP_num_spectrogram
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_ufp_num_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.3_l2_0.0001_higher_early_stopping.tf')
        file = 'audio_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_ufp_num_10s_ma_spec_label_raw')
        print("best ufp_num_spec_model loaded")
        
elif input_model == 'noise_image_model':
    with strategy.scope():
        # Load the best model for noise_image
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_noise_10s_ma_image_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,image_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.3_l2_0.0001.tf')
        file = 'image_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_noise_10s_ma_image_label_raw')
        print("best noise_image_model loaded")
        
elif input_model == 'noise_spec_model':
    with strategy.scope():
        # Load the best model for noise_spec
        pre_trained_model = load_model('rishabh_model_files/models/XGB_experiment_1/ln_noise_10s_ma_spec_label_raw,ConvNeXT_linear_AdamW_Base_L2_Dropout,audio_path,gp_v2_v3_Training_Only_Combined_IMAGINE_goproUFP_V2_V3_04252024_XGB_expt_4_dropout_0.3_l2_0.0001.tf')
        file = 'audio_path'
        train_generator, validate_generator, callbacks = get_train_val_generator('ln_noise_10s_ma_spec_label_raw')
        print("best noise_spec_model loaded")

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


Which pre-trained model do you want to load: ufp_size_image_model, ufp_size_spec_model, ufp_num_image_model, ufp_num_spec_model, noise_image_model, noise_spec_model ? noise_image_model
Found 18000 validated image filenames.
Found 4500 validated image filenames.
best noise_image_model loaded


In [ ]:
#Train the model

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
    model = pre_trained_model
    

#Fit Model
model.fit(train_generator,
          validation_data=validate_generator,
          epochs=10,
          steps_per_epoch=int(np.ceil(train_generator.samples/train_generator.batch_size)),
          validation_steps=int(np.ceil(validate_generator.samples/validate_generator.batch_size)),
          callbacks=callbacks
         )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


Epoch 1/10
INFO:tensorflow:Collective all_reduce tensors: 346 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 346 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 346 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa638bf3280>, 140352104729712), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1024,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa638bf3280>, 140352104729712), {}).


INFO:tensorflow:Assets written to: ./rishabh_model_files/models/data_estimation_experiment/set_3,ln_noise_10s_ma_image_label_raw,convnext,image_path,data_estimation_expt.tf/assets


INFO:tensorflow:Assets written to: ./rishabh_model_files/models/data_estimation_experiment/set_3,ln_noise_10s_ma_image_label_raw,convnext,image_path,data_estimation_expt.tf/assets


282/282 [==============================] - 449s 2s/step - loss: 0.0375 - mae: 0.1524 - val_loss: 0.0088 - val_mae: 0.0731 - lr: 1.0000e-04
Epoch 3/10
282/282 [==============================] - 406s 1s/step - loss: 0.0360 - mae: 0.1496 - val_loss: 0.0088 - val_mae: 0.0742 - lr: 1.0000e-04
Epoch 4/10
282/282 [==============================] - ETA: 0s - loss: 0.0359 - mae: 0.1491
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
282/282 [==============================] - 409s 1s/step - loss: 0.0359 - mae: 0.1491 - val_loss: 0.0090 - val_mae: 0.0762 - lr: 1.0000e-04
Epoch 5/10
282/282 [==============================] - 407s 1s/step - loss: 0.0350 - mae: 0.1478 - val_loss: 0.0098 - val_mae: 0.0799 - lr: 1.0000e-05
Epoch 6/10
282/282 [==============================] - ETA: 0s - loss: 0.0343 - mae: 0.1459
Epoch 6: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
282/282 [==============================] - 405s 1s/step - loss: 0.0343 - mae: 0.1459 - v

In [ ]:
########## Run predictions ##########

In [16]:
# Choose from the 3 different data splits (expt_set_number)
# expt_set = input("Which experiment set do you want to use: set_1, set_2 or set_3 ?")
print(f"Preparing test set for {expt_set} experiment")
if expt_set == 'set_1':
    filtered_metadata_test = metadata[metadata['expt_set_number'].isin(['set_2', 'set_3'])]
elif expt_set == 'set_2':
    filtered_metadata_test = metadata[metadata['expt_set_number'].isin(['set_1', 'set_3'])]
elif expt_set == 'set_3':
    filtered_metadata_test = metadata[metadata['expt_set_number'].isin(['set_1', 'set_2'])]
    
# Get the total count of the filtered rows
total_count = filtered_metadata_test.shape[0]

print(f"Total count of filtered rows: {total_count}")

Preparing test set for set_2 experiment
Total count of filtered rows: 27500


In [17]:
# Create a seperate CSV for test set

# Load the metadata
metadata = pd.read_csv('compiled data/rishabh/Data_estimation_experiments/test_site_metadata_site_id_2_with_data_splits.csv')

# Filter rows where expt_set_number is set_2 or set_3
filtered_metadata = metadata[metadata['expt_set_number'].isin(['set_1', 'set_3'])]

# Save the filtered dataset to a new CSV file
filtered_metadata.to_csv('compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/test_set_1_set_3_metadata.csv', index=False)

print("Filtered dataset saved to 'compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/test_set_1_set_3_metadata.csv'.")

Filtered dataset saved to 'compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/test_set_1_set_3_metadata.csv'.


In [22]:
# Load test CSV
test_metadata = pd.read_csv('compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/test_set_1_set_3_metadata.csv')
test_metadata[4990:5010].head(20)

,datetime,image_path,audio_path,ln_ufp_num_10s_ma_image_label_raw,ln_ufp_num_10s_ma_spec_label_raw,ln_ufp_size_10s_ma_image_label_raw,ln_ufp_size_10s_ma_spec_label_raw,ln_noise_10s_ma_image_label_raw,ln_noise_10s_ma_spec_label_raw,ufp_num_10s_ma_image_label_raw,...,image_extension,audio_extension,image_name,audio_name,pair_pm25,set_V3ext,set,vsby_nm,expt_set_number,train_val
4990,2022-03-18T01:01:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.908281,9.924544,3.871201,3.871201,4.114964,4.097838,20096.1,...,.jpg,.png,220317_2101.jpg,220317_2101.png,27.24600,trn,trn,15.0,set_1,val
4991,2022-03-18T01:02:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.921838,9.912477,3.871201,3.871201,4.141864,4.150725,20370.4,...,.jpg,.png,220317_2102.jpg,220317_2102.png,27.24600,tst,tst,15.0,set_1,val
4992,2022-03-18T01:03:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.981064,10.009630,3.881564,3.891820,4.156380,4.139796,21613.3,...,.jpg,.png,220317_2103.jpg,220317_2103.png,27.24600,val,val,15.0,set_1,val
4993,2022-03-18T01:04:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.910175,9.953006,3.889777,3.875359,4.151040,4.162315,20134.2,...,.jpg,.png,220317_2104.jpg,220317_2104.png,27.24600,tst,tst,15.0,set_1,val
4994,2022-03-18T01:05:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.018007,9.935132,3.843744,3.856510,4.086312,4.116595,22426.7,...,.jpg,.png,220317_2105.jpg,220317_2105.png,27.24600,trn,trn,15.0,set_1,val
4995,2022-03-18T01:06:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.462154,9.951272,4.300003,3.916015,4.126973,4.109069,34966.8,...,.jpg,.png,220317_2106.jpg,220317_2106.png,27.24600,val,val,15.0,set_1,val
4996,2022-03-18T01:07:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.873142,9.985763,3.885679,3.843744,4.178686,4.147253,19402.2,...,.jpg,.png,220317_2107.jpg,220317_2107.png,27.24600,trn,trn,15.0,set_1,val
4997,2022-03-18T01:08:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.891304,9.862207,3.871201,3.877432,4.071417,4.118224,19757.8,...,.jpg,.png,220317_2108.jpg,220317_2108.png,27.24600,tst,tst,15.0,set_1,val
4998,2022-03-18T01:09:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.900373,9.900979,3.887730,3.879500,4.155910,4.133886,19937.8,...,.jpg,.png,220317_2109.jpg,220317_2109.png,27.24600,trn,trn,15.0,set_1,val
4999,2022-03-18T01:10:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.876810,9.903173,3.871201,3.873282,4.096010,4.178686,19473.5,...,.jpg,.png,220317_2110.jpg,220317_2110.png,27.24600,trn,trn,15.0,set_1,val


In [27]:
# Run predictions on the entire test dataset - set_id data

#Define Preprocessing functions
architecture_preprocessing_convnext = K.applications.convnext.preprocess_input

#Define Test Generators

generator_test_convnext = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing_convnext, 
                                                     horizontal_flip=False,
                                                     vertical_flip = False)

test_generator_convnext_audio = generator_test_convnext.flow_from_dataframe(dataframe=test_metadata[['audio_path']].reset_index(drop=True),
                                                     x_col= 'audio_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

test_generator_convnext_image = generator_test_convnext.flow_from_dataframe(dataframe=test_metadata[['image_path']].reset_index(drop=True),
                                                     x_col= 'image_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

#################################### Run predictions on models ####################################################

# Best model for log(ufp_num) based on images:  ln_ufp_num, image, convnext, raw
print("running predictions for ln_ufp_num_convnext_image_raw_model")
ln_ufp_num_convnext_image_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_ufp_num_10s_ma_image_label_raw,convnext,image_path,data_estimation_expt.tf')
test_metadata['ln_ufp_num_convnext_image_raw_prediction'] = ln_ufp_num_convnext_image_raw_model.predict(x=test_generator_convnext_image, 
                                                                                           steps=int(np.ceil(test_generator_convnext_image.samples/test_generator_convnext_image.batch_size)))
print("running predictions for ln_ufp_num_convnext_audio_raw_model")
# Best model for log(ufp_num) based on audio: ln_ufp_num, audio, convnext, raw
ln_ufp_num_convnext_audio_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_ufp_num_10s_ma_spec_label_raw,convnext,audio_path,data_estimation_expt.tf')
test_metadata['ln_ufp_num_convnext_audio_raw_prediction'] = ln_ufp_num_convnext_audio_raw_model.predict(x=test_generator_convnext_audio, 
                                                                                             steps=int(np.ceil(test_generator_convnext_audio.samples/test_generator_convnext_audio.batch_size)))
print("running predictions for ln_ufp_size_convnext_image_raw_model")
# Best model for log(ufp_size) based on images: ln_ufp_size, image, convnext, raw 
ln_ufp_size_convnext_image_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_ufp_size_10s_ma_image_label_raw,convnext,image_path,data_estimation_expt.tf')
test_metadata['ln_ufp_size_convnext_image_raw_prediction'] = ln_ufp_size_convnext_image_raw_model.predict(x=test_generator_convnext_image, 
                                                                                             steps=int(np.ceil(test_generator_convnext_image.samples/test_generator_convnext_image.batch_size)))
print("running predictions for ln_ufp_size_convnext_audio_raw_model")
# Best model for log(ufp_size) based on audio: ln_ufp_size, audio, convnext, raw
ln_ufp_size_convnext_audio_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_ufp_size_10s_ma_spec_label_raw,convnext,audio_path,data_estimation_expt.tf')
test_metadata['ln_ufp_size_convnext_audio_raw_prediction'] = ln_ufp_size_convnext_audio_raw_model.predict(x=test_generator_convnext_audio,
                                                                                                     steps=int(np.ceil(test_generator_convnext_audio.samples/test_generator_convnext_audio.batch_size)))
print("running predictions for ln_noise_convnext_image_raw_model")
# Best model for log(noise) based on images: ln_noise, image, convnext, raw
ln_noise_convnext_image_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_noise_10s_ma_image_label_raw,convnext,image_path,data_estimation_expt.tf')
test_metadata['ln_noise_convnext_image_raw_prediction'] = ln_noise_convnext_image_raw_model.predict(x=test_generator_convnext_image,
                                                                                               steps=int(np.ceil(test_generator_convnext_image.samples/test_generator_convnext_image.batch_size)))
print("running predictions for ln_noise_convnext_audio_raw_model")
# Best model for log(noise) based on audio: ln_noise, audio, convnext, raw
ln_noise_convnext_audio_raw_model = load_model('rishabh_model_files/models/data_estimation_experiment/set_2,ln_noise_10s_ma_spec_label_raw,convnext,audio_path,data_estimation_expt.tf')
test_metadata['ln_noise_convnext_audio_raw_prediction'] = ln_noise_convnext_audio_raw_model.predict(x=test_generator_convnext_audio, 
                                                                                             steps=int(np.ceil(test_generator_convnext_audio.samples/test_generator_convnext_audio.batch_size)))               

# Save dataframe with all predictions added
test_metadata.to_csv(path_or_buf='compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/set_1_3_predictions_added.csv', index=False)


Found 27500 validated image filenames.
Found 27500 validated image filenames.
running predictions for ln_ufp_num_convnext_image_raw_model
430/430 [==============================] - 473s 1s/step
running predictions for ln_ufp_num_convnext_audio_raw_model
430/430 [==============================] - 355s 808ms/step
running predictions for ln_ufp_size_convnext_image_raw_model
430/430 [==============================] - 473s 1s/step
running predictions for ln_ufp_size_convnext_audio_raw_model
430/430 [==============================] - 352s 801ms/step
running predictions for ln_noise_convnext_image_raw_model
430/430 [==============================] - 479s 1s/step
running predictions for ln_noise_convnext_audio_raw_model
430/430 [==============================] - 357s 813ms/step


In [28]:
test_met = pd.read_csv('compiled data/rishabh/Data_estimation_experiments/dataset_split_2_experiment_predictions/set_1_3_predictions_added.csv')
test_met.head(10)

,datetime,image_path,audio_path,ln_ufp_num_10s_ma_image_label_raw,ln_ufp_num_10s_ma_spec_label_raw,ln_ufp_size_10s_ma_image_label_raw,ln_ufp_size_10s_ma_spec_label_raw,ln_noise_10s_ma_image_label_raw,ln_noise_10s_ma_spec_label_raw,ufp_num_10s_ma_image_label_raw,...,set,vsby_nm,expt_set_number,train_val,ln_ufp_num_convnext_image_raw_prediction,ln_ufp_num_convnext_audio_raw_prediction,ln_ufp_size_convnext_image_raw_prediction,ln_ufp_size_convnext_audio_raw_prediction,ln_noise_convnext_image_raw_prediction,ln_noise_convnext_audio_raw_prediction
0,2022-01-28T19:46:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.460771,8.400569,3.238678,3.222868,4.174695,4.296741,4725.7,...,trn,30.0,set_1,train,9.503922,9.648796,2.865328,2.862480,4.200672,4.202183
1,2022-01-28T19:47:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.119160,8.125069,3.310543,3.284664,4.036715,4.187835,3358.2,...,trn,30.0,set_1,train,9.377732,9.014420,2.836235,2.875058,4.091931,4.057718
2,2022-01-28T19:48:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.036897,7.893759,3.306887,3.430756,3.980055,3.910422,3093.0,...,val,30.0,set_1,train,9.551662,9.902980,2.829417,2.913443,4.154562,4.232822
3,2022-01-28T19:49:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,7.991119,8.082618,3.367296,3.265759,3.933784,3.924347,2954.6,...,trn,30.0,set_1,train,9.934647,9.784182,2.850960,2.887939,4.096005,4.184093
4,2022-01-28T19:50:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,8.069468,8.170836,3.222868,3.222868,4.134686,4.149937,3195.4,...,val,30.0,set_1,train,9.493560,9.685939,2.701426,2.874400,4.083997,4.087193
5,2022-01-28T19:51:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.328667,10.172396,3.218876,3.218876,4.242477,4.192529,30597.3,...,trn,30.0,set_1,train,9.639635,9.865390,2.662848,2.851434,4.159563,4.203203
6,2022-01-28T19:52:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,10.082575,9.962553,2.639057,2.602690,4.083452,4.087656,23922.5,...,trn,30.0,set_1,train,9.687941,9.731537,2.748302,2.835529,4.057446,4.019667
7,2022-01-28T19:53:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.408543,9.307331,2.624669,2.694627,4.163093,4.145671,12192.1,...,trn,30.0,set_1,train,9.607394,9.857638,2.969624,2.986380,4.156274,4.247086
8,2022-01-28T19:54:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.872606,9.844661,2.833213,2.833213,4.177919,4.171460,19391.8,...,trn,30.0,set_1,train,9.507451,9.288246,2.614384,2.955821,4.095201,4.203938
9,2022-01-28T19:55:00Z,data/version_2/Site_2_Mistall_28/Images/28_220...,data/version_2/Site_2_Mistall_28/Spectrograms/...,9.503771,9.560391,2.821379,2.734368,4.142817,4.163871,13410.2,...,trn,30.0,set_1,train,9.515207,9.597008,2.634300,3.040588,4.108944,4.070339


In [26]:
import os

# Define the list of file paths
file_paths = [
    'rishabh_model_files/models/data_estimation_experiment/set_2,ln_ufp_num_10s_ma_spec_label_raw,convnext,audio_path,data_estimation_expt.tf'
]

# Iterate over the list and check if each file path exists
for file_path in file_paths:
    if os.path.exists(file_path):
        print("All files exist")
    else:
        print(f"The file path does not exist: {file_path}")

All files exist
